In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# unscaled, log-normalized counts, with conditions subsampled to the same number of cells 
# and 2000 highly variable genes calculated jointly across all perturbation conditions, including control, using scanpy28 with default parameters (Supplementary Methods)

In [3]:
from anndata import read_h5ad
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd 
import scanpy as sc
import seaborn as sns
import string
import sys
sys.path.append("../../scxmatch/src/")
from scxmatch import *
np.random.seed(42)

In [4]:
reference = 0
group_by = "dose_value"

In [5]:
df = pd.concat([pd.read_csv(f"../evaluation_results/sciplex_MCF7_matching_properties_{group}.txt", delimiter=", ").drop(["p", "z"], axis=1) for group in [10, 100, 1000, 10000]])

/tmp/ipykernel_3803311/3940062803.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.concat([pd.read_csv(f"../evaluation_results/sciplex_MCF7_matching_properties_{group}.txt", delimiter=", ").drop(["p", "z"], axis=1) for group in [10, 100, 1000, 10000]])
/tmp/ipykernel_3803311/3940062803.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.concat([pd.read_csv(f"../evaluation_results/sciplex_MCF7_matching_properties_{group}.txt", delimiter=", ").drop(["p", "z"], axis=1) for group in [10, 100, 1000, 10000]])
/tmp/ipykernel_3803311/3940062803.py:1: ParserWarning: Falling back to the 'python

In [6]:
df.loc[df["k"].isna(), "k"] = df.loc[df["k"].isna(), "#nodes"] - 1

In [7]:
df.rename({"test": "dose value", "s": "rel. support"}, axis=1, inplace=True)

In [8]:
groups = sorted(df["dose value"].unique())

In [9]:
colors = sns.color_palette("hls", len(groups) + 1)
pal = {group: colors[i] for i, group in enumerate(groups + [reference])}

In [10]:
melted = pd.melt(df[df["#nodes"] >= 1000], id_vars=["k", "ref", "dose value", "#nodes"], var_name="metric")

In [ ]:
g = sns.relplot(melted, x="k", y="value", row="metric", hue="dose value", col="#nodes", facet_kws={"sharex": False, "sharey":False}, kind="line", palette=pal, marker="o", linestyle="dashed", height=3, aspect=1)
sns.move_legend(g, loc="upper center", ncol=4, bbox_to_anchor=(0.5, 1.05))
    
labels = string.ascii_lowercase
for i in range(len(melted["metric"].unique())):
    g.axes[i, 0].text(
        -0.05, 1.2,  # Position (normalized figure coordinates)
        labels[i],   # Corresponding letter
        transform=g.axes[i, 0].transAxes,  # Relative to subplot
        fontsize=10, fontweight='bold', va='top', ha='left'
    )
plt.savefig("../plots/matching_weights.pdf", bbox_inches="tight")